# Practical 7 - Part 2A(Mingzhou Hu)
This project explores the geometry of a single camera. The aim is to take several points on
a plane, and predict where they will appear in the camera image. Based on these observed
points, we will then try to re-estimate the Euclidean transformation relating the plane and
the camera. In practical 2B we will use this code to draw a wireframe cube
on an augmented reality marker.   You should use this
template for your code and fill in the missing sections marked "TO DO"

# Import libraries 

In [1]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

# TO DO - fill in projectiveCamera and estimatePlanePose functions  (you will have to utilise your solutions for solveAXEqualsZero and calcBestHomography from Part 1)

In [2]:
def solveAXEqualsZero(A):
    # TO DO: Write this routine - it should solve Ah = 0
    # A=UL(V.T)
    # Suppose A is a i by j matrix, the u will be a i by i matrix, s will be a i by j matrix and
    # v will be a j by j matrix.
    u,s,v=np.linalg.svd(A)
    # Take last column of v
    v=v.T
    h=v[:,-1]
    return h

In [3]:
# This function should apply the direct linear transform (DLT) algorithm to calculate the best 
# homography that maps the points in pts1Cart to their corresonding matching in pts2Cart
def calcBestHomography(pts1Cart, pts2Cart):    
    # TO DO: replace this
    H = []

    # TO DO: 
    # First convert points into homogeneous representation
    pts1Hom = np.concatenate((pts1Cart, np.ones((1,pts1Cart.shape[1]))), axis=0)
    pts2Hom = np.concatenate((pts2Cart, np.ones((1,pts2Cart.shape[1]))), axis=0)
    
    # Then construct the matrix A, size (n_points,9)
    n_points=2*pts1Hom.shape[1]
    A=np.zeros((n_points,9))
    for i in range(5):
        u_v_1=pts1Hom[:,i]
        x_y_1=pts2Hom[:,i]
        A[2*i,:]=np.hstack(([0,0,0],-u_v_1,x_y_1[1]*u_v_1))
        A[2*i+1,:]=np.hstack((u_v_1,[0,0,0],-x_y_1[0]*u_v_1))
    
    # Solve Ah = 0
    h=solveAXEqualsZero(A)

    # Reshape h into the matrix H, values of h go first into rows of H
    H=np.reshape(h,(3,3))
    
    return H

In [4]:
#The goal of this function is to project points in XCart through projective camera
#defined by intrinsic matrix K and extrinsic matrix T.
def projectiveCamera(K,T,XCart):
    
    # TO DO: Replace this
    # XImCart =

    # TO DO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    XHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    
    # TO DO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    XCamHom=np.matmul(T,XHom)
    
    # TO DO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    XCamHom_normal=XCamHom[:-1,:]
    
    # TO DO: Move points to image coordinates xImHom by applying intrinsic matrix
    XImHom=np.matmul(K,XCamHom_normal)
    
    # TO DO: Convert points back to Cartesian coordinates xImCart
    XImCart=XImHom[0:2,:] / np.tile([XImHom[2,:]],(2,1))
    
    return XImCart


In [5]:
# Goal of function is to estimate pose of plane relative to camera (extrinsic matrix)
# given points in image xImCart, points in world XCart and intrinsic matrix K

def estimatePlanePose(XImCart,XCart,K):

    # TO DO: replacH[:,:2]e this
    # T = 

    # TO DO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    XImHom = np.concatenate((XImCart, np.ones((1,XImCart.shape[1]))), axis=0)
    
    # TO DO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    XCamHom=np.matmul(np.linalg.inv(K),XImHom)
    
    # TO DO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    XCamCart = XCamHom[0:2,:] / np.tile([XCamHom[2,:]],(2,1))
    
    XCart=XCart[:-1,:]
    H=calcBestHomography(XCart, XCamCart)
          
    # TO DO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD 
    u,s,v=np.linalg.svd(H[:,:2])
    L=np.array([[1,0],
                [0,1],
                [0,0]])
    # U*L*V.T
    R12=np.matmul(np.matmul(u,L),v)


    # TO DO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R3=np.cross(R12[:,0],R12[:,1])
    R3=R3.reshape((3,1))
   
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    R=np.hstack((R12,R3))
    if np.linalg.det(R)<=0:
        R[:,-1]=-1*R[:,-1]

    # TO DO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    k=np.sum(H[:,:2]/R[:,:2])/6
    t=H[:,2]/k 
    t=t.reshape((3,1))
    
    # TO DO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[2,:] < 0:
        t= t*-1
        R[:,:2] = R[:,:2] * -1
            
    # TO DO: Assemble transformation into matrix form
    T=np.hstack((R,t))

    return T 

# Once you have completed these functions, use them to estimate the transformation from the plane co-ordinate system to the camera co-ordinate system (i.e. the extrinsic matrix)

In [6]:
# We assume that the intrinsic camera matrix K is known and has values
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are four known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
  
# TO DO: Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera" - body of function appears below:

XImCart=projectiveCamera(K,T,XCart)

# TO DO: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
XImCart=XImCart+np.random.normal(0, 3, XImCart.shape)


# TO DO: Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function "estimate plane pose"
TEst=estimatePlanePose(XImCart,XCart,K)

# If you have got this correct, Test should closely resemble T above

In [7]:
T

array([[ 9.8510e-01, -4.9200e-02,  1.6190e-01,  4.6000e+01],
       [-1.6230e-01, -5.5200e-01,  8.1810e-01,  7.0000e+01],
       [ 4.9000e-02, -8.3240e-01, -5.5180e-01,  5.0089e+02],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])

In [8]:
TEst

array([[ 9.87848111e-01, -1.38459646e-01,  7.06047933e-02,
         3.97009137e+01],
       [-1.36327639e-01, -5.53759388e-01,  8.21440999e-01,
         6.47564499e+01],
       [-7.46383630e-02, -8.21084324e-01, -5.65906042e-01,
         4.54275818e+02]])

The goal in this practice is to estimate extrinsic matrix. If the values in TEst are closer to those in T, it can illustrate that the estimation is more accurate. We can see from the above outputs, it clear that TEst and T are very similar, so our estimation is accurate.